<div class="well" style="margin:1em 2em">
<p>This Notebook reproduces and expands on a demo from “Distant Reading of Direct Speech in Epic: An Illustrated Workflow,” a talk I gave at the FIEC / CA annual meeting in London, July 8, 2019.</p>
</div>


# Heroes and their moms

Let's say we're young scholars interested in Telemachus' speech to Penelope.
 - How often does he speak to her?
 - What kind of language does he use?
 - How does the narrator refer to these speeches?
 
We'll start by showing how the DICES database and Python library can be used to retrieve and manipulate the speeches in question. Then we'll expand our perspective to show how DICES enables research on a "distant reading" scale, taking in all heroes and their mothers. Finally, we'll check the accuracy of the automated methods by comparing against a benchmark of hand-curated mother-child speech data.

## Preliminaries

In [ ]:
# this lets me change the api while the notebook is open
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import re
import ipywidgets as widgets
from IPython.display import display
from collections import Counter
from matplotlib import pyplot
%matplotlib inline

### The DICES API

See example 1 for notes.

In [ ]:
from dicesapi import DicesAPI
api = DicesAPI(
    dices_api = 'http://localhost:8000/api',
    cts_api = 'http://cts.perseids.org/api/cts/',
)

### CLTK

In [ ]:
from cltk.tokenize.word import WordTokenizer
tokenizer = {
    'greek': WordTokenizer('greek'),
    'latin': WordTokenizer('latin'),
}

from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
from cltk.lemmatize.greek.backoff import BackoffGreekLemmatizer
lemmatizer = {
    'greek': BackoffGreekLemmatizer(),
    'latin': BackoffLatinLemmatizer(),    
}

# regular expressions to tidy up perseus texts for ctlk
replacements = {
    'greek': [
        (r'·', ','),           # FIXME: raised dot? 
        (chr(700), chr(8217)), # two different apostrophes that look alike
    ],
    'latin': [
        
    ],
}

# compile the regexes
for lang in ['greek', 'latin']:
    replacements[lang] = [(re.compile(pat), repl) for pat, repl in replacements[lang]]
    

# generic tokenize-lemmatize function
def lemmatize(text, lang):
    '''return a set of (token,lemmata) pairs for a string'''
    
    for pat, repl in replacements[lang]:
        text = pat.sub(repl, text)
    
    tokens = tokenizer[lang].tokenize(text)
    lemmata = lemmatizer[lang].lemmatize(tokens)
    
    return lemmata

### WikiData

In [ ]:
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.entity import WikidataItem, WikidataProperty

##  Part 1

Let's start by building a lexicon for all the words Telemachus speaks to Penelope.

### Identify and download the speeches

Using the hand-rolled DICES API code, we can search speeches using keywords. For now, JSON results from the API are paged, so if your search has a lot of results, you may have to wait for several pages to download. I've added a progress bar widget because I get impatient.

Note that I can specify both the speaker and the addressee.

In [ ]:
speeches = api.getSpeeches(spkr_name='Telemachus', addr_name='Penelope')

What did we get?

In [ ]:
for s in speeches:
    print(s)

### Retrieve the passages from a remote library

We have the metadata for each speech; now we need the text. The DICES library uses MyCapytain under the hood to retrieve the passages from a remote CTS server: here, Perseus, specified in the `DicesAPI()` call above.

In [ ]:
passages = []
for s in speeches:
    cts_passage = s.getCTS()
    text = cts_passage.text
    passages.append(text)
    
    print(f'{s.author.name} {s.work.title} {s.l_range}')
    print(text)
    print()

### Use CLTK to parse the text

We can use CTLK's tokenizers to break each string into meaningful units -- sentences and/or words. Then we use the backoff lemmatizer to normalize all the inflected forms to dictionary headwords.

I rolled these steps into one convenience function up above. 👉🏻 *One thing to watch out for is that CTLK needs to know what language you're working with, so I've added a kludge to set language based on the author name; really, that should be built into DICES eventually.*

In [ ]:
lems = Counter()
for p in passages:
    lang = s.getLang()
    lemmatized = lemmatize(p.lower(), lang)
    
    these_lems = [lem for tok, lem in lemmatized]
    lems.update(these_lems)

Convert the counter to a Pandas data frame for tidier presentation.

In [ ]:
results = pd.DataFrame(lems.most_common(), columns=['lemma', 'count'])
results

## Part 2

Now let's think more broadly. How typical is this kind of speech? We can use external linked data to find other examples of mother-son conversations in the corpus.

### Some custom code to query WikiData

This lets us ask whether a given addressee belongs to the set of people having a certain relationship to a given speaker. It takes a while to download the WikiData entities, and I had to run this a number of times, so I cached WD data in the respective character objects once it's downloaded.

In [ ]:
def checkWD(c):
    '''make sure character has wikidata id'''
    if c.char is not None:
        if c.char.wd is not None:
            if len(c.char.wd.strip()) > 0:
                return c.char.wd.strip()

def checkWDRelation(s, a, relation, cache=None):
    if cache is None:
        cache = {}
    else:
        if (s.id, a.id) in cache:
            return cache[(s.id, a.id)]

    res = False

    if not hasattr(s, 'wd_ent'):
        s.wd_ent = WikidataItem(get_entity_dict_from_api(s.wd))

    claim_group = s.wd_ent.get_truthy_claim_group(relation)

    for claim in claim_group:
        if claim.mainsnak.datavalue is None:
            continue
        if claim.mainsnak.datavalue.value['id'] == a.wd:
            res = True
    
    cache[(s.id, a.id)] = res
    return res

For example, the relation "mother of" has the WikiData ID `'P25'`. Here's how we ask if a given addressee is the mother of a given speaker:

In [ ]:
speaker = api.getCharacters(name='Telemachus')[0]
addressee = api.getCharacters(name='Penelope')[0]

print(f'Is {addressee.name} the mother of {speaker.name}?')
print(checkWDRelation(speaker, addressee, 'P25'))

I also added a separate cache just for the boolean result of checkWDRelation, to save a little more time.

In [ ]:
cache_mothers = {}

### Using WikiData to filter the speeches

The DICES dataset includes WikiData ids for most of the characters (not all). The DICES API doesn't let us query WikiData itself, though. For now, the easiest thing for now is just to download all the speeches and character IDs, and then cross reference them against WikiData using its own API.

In [ ]:
# download all the speeches: takes a minute
speeches = api.getSpeeches(progress=True)

**Check each speaker-addressee pair against WikiData**

What we actually do here is download the WikiData entity for each speaker, if we don't already have it cached. Then we ask the WD entity for it's mom(s), and check the WD ID of the addressee against the results.

In [ ]:
df = []

# create a progress bar
pbar = widgets.IntProgress(
    value = 0,
    min = 0,
    max = len(speeches),
    bar_style='info',
    orientation='horizontal'
)
pbar_label = widgets.Label(value = f'{pbar.value}/{len(speeches)}')
display(widgets.HBox([pbar, pbar_label]))

for s in speeches:
    if s.spkr is not None and s.addr is not None:
        for spkr in s.spkr:
            spkr_wd = checkWD(spkr)
            if spkr_wd is not None:

                for addr in s.addr:
                    addr_wd = checkWD(addr)
                    if addr_wd is not None:
                        df.append((
                            s.id,
                            s.work.title,
                            s.l_fi,
                            s.l_la,
                            spkr.char.name, spkr_wd, 
                            addr.char.name, addr_wd,
                            checkWDRelation(spkr.char, addr.char, 'P25', cache=cache_mothers),
                            checkWDRelation(addr.char, spkr.char, 'P25', cache=cache_mothers)
                            ))
    pbar.value += 1
    pbar_label.value = f'{pbar.value}/{len(speeches)}'

df = pd.DataFrame(df, columns=['id', 'work', 'l_first', 'l_last', 'spkr', 'sp_wd', 'addr', 'ad_wd', 'sp_is_mom', 'ad_is_mom'])

🤔 Let's take a look at the results. Here is the complete set of speeches, with the additional attribute `sp_is_mom` if the speaker is the addressee's mother, and `ad_is_mom` if the addressee is the speaker's mother.

As a quick sanity check, the first two speeches in the Argonautica, which were at the top of the list when I ran this, are between Jason and his mother, Alcimede.

In [ ]:
df

Thanks to pandas, we can filter the data frame on the new boolean columns to show only speeches between mother and child.

In [ ]:
hits = df.loc[df['sp_is_mom'] | df['ad_is_mom'],
             ['work', 'l_first', 'l_last', 'spkr', 'addr']]
hits

Pandas also comes in handy if I wanted to export this data to Excel:

In [ ]:
df.to_csv('example.csv')

### Validation

Let's see how well the automated approach worked. We'll load up a hand-corrected list of mother-child speeches and compare.

In [ ]:
bench = pd.read_csv('data/moms-bench.csv', dtype=str)
bench

Let's look at the union of `hits` and `bench` to see how we did:

In [ ]:
results = hits.merge(bench, on=['work', 'l_first'], how='outer', 
                        suffixes=['_h', '_b'], indicator=True)
results[['work', 'l_first', 'spkr_h', 'addr_h', 'spkr_b', 'addr_b', '_merge']]

#### Precision and Recall

In [ ]:
true_pos = sum(results['_merge'] == 'both')

p = true_pos / hits.shape[0]
r = true_pos / bench.shape[0]

print(f'Precision: {p:.2f}')
print(f'Recall:    {r:.2f}')

### Discussion

#### The good news
Well, we returned no false positives, and managed to catch 80% of the benchmark set.

#### The bad news
Lets' look a little more closely at the speeches we missed:

In [ ]:
missed = results[results['_merge'] == 'right_only'][
                ['work', 'l_first', 'spkr_h', 'addr_h', 'spkr_b', 'addr_b', 'notes']]
missed

At a glance, I'd say these fall into two groups:

 1. A conversation in the Iliad between Hera and a group of gods, some of whom were here children
 2. Conversations in the Aeneid between Venus and Aeneas
 
Missing the first group seems somewhat understandable, if the addressee wasn't explicitly named. Missing the second group, on the other hand, is a big problem!

#### Digging a little deeper

First, let's confirm that all these speeches are in the database results.

In [ ]:
missed.merge(df, how='left', on=['work', 'l_first'])[[
    'work', 'l_first',                               # keys: work and locus
    'id', 'spkr', 'addr', 'sp_is_mom', 'ad_is_mom',  # cols from df
    'spkr_b', 'addr_b'                               # cols from bench
    
]].groupby('id').agg(lambda x: ', '.join(x.unique())) # for speeches with multiple speakers/
                                                      # addressees: combine on speech id

Yep, all the missed speeches were in the database. The ones involving Hera's children as a collective 'gods' clearly would be difficult to match with this method.

But what happened with Aeneas and Venus?? WikiData must not treat Venus as Aeneas' mother...?

In [ ]:
speaker = api.getCharacters(name='Aeneas')[0]
addressee = api.getCharacters(name='Venus')[0]

print(f'Is {addressee.name} the mother of {speaker.name}?')
print(checkWDRelation(speaker, addressee, 'P25'))

So after some head scratching, the problem turns out to be that WikiData has separate entries for [Venus](https://www.wikidata.org/wiki/Q47652) and [Aphrodite](https://www.wikidata.org/wiki/Q35500). Only the latter is listed as a mother of [Aeneas](https://www.wikidata.org/wiki/Q82732).

In [ ]:
speaker = api.getCharacters(name='Aeneas')[0]
addressee = api.getCharacters(name='Aphrodite')[0]

print(f'Is {addressee.name} the mother of {speaker.name}?')
print(checkWDRelation(speaker, addressee, 'P25'))

### Takeaways

 - Part of the fault here is ours: the database underlying DICES betrays its diverse origins by a lingering heterogeneity. If we think Aphrodite and Venus are the same character, we'd better make sure we refer to her the same way consistently, or risk weird results.
 
 - WikiData gave us a lot for free -- all of the individual mother-child relationships were in there when we knew where to look.
 
 - But missing Venus was a pretty big "gotcha" in the end.
 
 - If we want to rely on linked open data for high-stakes work, we need resources that are sensitive to the details we care about. We hope that MANTO, because it's specific to Classical myth and hand-curated by domain experts, can help us with problems like when to treat Venus and Aphrodite as independent entities and when to consider them identical.